#### Data Manipulation

You should do substantive work on at least six subsets of the data. 

- 3 sets of 10% of the data from the UCI Madelon set
- 3 sets of 10% of the data from the Madelon set made available by your instructors (20000 rows, 1001 columns)

##### Jupyter Notebook, EDA 

- perform EDA on each set as you see necessary

In [22]:
# !conda install psycopg2 --yes
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
%matplotlib inline
# !pip install tqdm
from tqdm import tqdm
import pickle


In [23]:
# import UCI madelon data and set the nan column to labels

def import_and_labels(data, labels):
    
    data = data
    labels = labels
    madelon_train = pd.read_csv(data, delimiter=' ', header=None)
    madelon_labels = pd.read_csv(labels, delimiter=' ', header=None)
    madelon_train[500] = madelon_labels

    return madelon_train

In [24]:
madelon_train = import_and_labels('madelon_train.data.csv','madelon_train.labels.csv')

In [25]:
# generate 3 random samples of 10% each from UCI Madelon

np.random.seed(42)
UCIsample1 = madelon_train.sample(200)
UCIsample2 = madelon_train.sample(200)
UCIsample3 = madelon_train.sample(200)

In [ ]:
# pull 3 random samples of 1% each from the class madelon database

def con_cur_to_class_db():
    con = pg2.connect(host='34.211.227.227',
                  dbname='postgres',
                  user='postgres')
    cur = con.cursor(cursor_factory=RealDictCursor)
    return con, cur

def draw_sample():
    con, cur = con_cur_to_class_db()
    cur.execute('SELECT * FROM madelon ORDER BY random() LIMIT 3000;')
    mad_db = cur.fetchall()
    con.close()
    return pd.DataFrame(mad_db)

In [ ]:
# DBsample = draw_sample()
# pickle.dump(DBsample, open('DBsample', "wb" ) )

In [26]:
DBsample = pickle.load( open( "DBsample", "rb" ) )

# drop the seemingly unneeded ID column
DBsample.drop('_id', axis=1, inplace=True)
DBsample.shape

(3000, 1001)

In [27]:
DBsample1 = DBsample.iloc[0:1000,:]
DBsample2 = DBsample.iloc[1000:2000,:]
DBsample3 = DBsample.iloc[2000:3001,:]

In [ ]:
# use decisiontree and kneighborsregression to determine noise in the samples

def calculate_r_2_for_feature(data, feature, regression_method):
    new_data = data.drop(feature, axis=1)

    X_train, X_test, y_train, y_test = train_test_split(new_data,data[feature],test_size=0.25)

    regressor = regression_method
    regressor.fit(X_train,y_train)

    score = regressor.score(X_test,y_test)
    return score

def mean_r2_for_feature(data, feature, regression_method):
    scores = []
    for _ in range(3):
        scores.append(calculate_r_2_for_feature(data, feature, regression_method))
    scores = np.array(scores)
    if scores.mean() > 0:
        return(feature)
    else:
        return 0

def return_features(data, regression_method):
    feature_list = []
    for i in range(len(data.columns)):
        feature_list.append(mean_r2_for_feature(data, i, regression_method))
    return feature_list

In [ ]:
# due to the time it took to run, I ran these and saved the results in a list shown later below
# feature_list_KN1 = return_features(UCIsample1, KNeighborsRegressor())
# feature_list_KN2 = return_features(UCIsample1, DecisionTreeRegressor())
# feature_list_KN3 = return_features(UCIsample2, KNeighborsRegressor())
# feature_list_DT1 = return_features(UCIsample2, DecisionTreeRegressor())
# feature_list_DT2 = return_features(UCIsample3, KNeighborsRegressor())
# feature_list_DT3 = return_features(UCIsample3, DecisionTreeRegressor())

In [ ]:
# minor changes had to be made to the functions to allow them to process Josh's Madelon samples w/ different indexes

def calculate_r_2_for_feature(data, feature, regression_method):
    new_data = data.drop(feature, axis=1)

    X_train, X_test, y_train, y_test = train_test_split(new_data,data[feature],test_size=0.25)

    regressor = regression_method
    regressor.fit(X_train,y_train)

    score = regressor.score(X_test,y_test)
    return score

def mean_r2_for_feature(data, feature, regression_method):
    scores = []
    for _ in range(3):
        scores.append(calculate_r_2_for_feature(data, feature, regression_method))
    scores = np.array(scores)
    if scores.mean() > 0:
        return(feature, scores.mean())

def return_features(data, regression_method):
    feature_list = []
    for i in tqdm(data.columns):
        feature_list.append((i, mean_r2_for_feature(data, i, regression_method)))
    return feature_list

In [ ]:
# relevant features saved below
# feature_list_KN1 = return_features(DBsample1.sample(200), KNeighborsRegressor())
# feature_list_KN2 = return_features(DBsample2.sample(200), KNeighborsRegressor())
# feature_list_KN3 = return_features(DBsample3.sample(200), KNeighborsRegressor())
# feature_list_DT1 = return_features(DBsample1.sample(200), DecisionTreeRegressor())
# feature_list_DT2 = return_features(DBsample2.sample(200), DecisionTreeRegressor())
# feature_list_DT3 = return_features(DBsample3.sample(200), DecisionTreeRegressor())

In [34]:
# eliminate noise in the samples
# here are the feature lists output by the R2 method.  note that more than 20 results showed up for several of the DB
# sample lists, so I included them all here with the expectation that further feature selection and modeling will 
# eliminate them

pickle.dump(UCIsample1, open('UCIsample1', "wb" ) )
pickle.dump(UCIsample2, open('UCIsample2', "wb" ) )
pickle.dump(UCIsample3, open('UCIsample3', "wb" ) )
pickle.dump(DBsample1, open('DBsample1', "wb" ) )
pickle.dump(DBsample2, open('DBsample2', "wb" ) )
pickle.dump(DBsample3, open('DBsample3', "wb" ) )

UCIsample_list = [28,48,64,105,128,153,241,281,318,336,338,378,433,442,451,453,455,472,475,493,500]
DBsample_list = ['feat_257','feat_269','feat_308','feat_315',\
                 'feat_336','feat_341','feat_395','feat_504','feat_526','feat_639','feat_681',\
                 'feat_701','feat_724','feat_736','feat_769','feat_808','feat_829','feat_867','feat_920',\
                 'feat_956','target']

def drop_noise(df, signal_list):
    temp_df = df
    for column in temp_df.columns:
        if column not in signal_list:
            temp_df.drop(column, axis=1, inplace=True)
    return temp_df

In [35]:
UCIsample1_clean = drop_noise(UCIsample1, UCIsample_list)
UCIsample2_clean = drop_noise(UCIsample2, UCIsample_list)
UCIsample3_clean = drop_noise(UCIsample3, UCIsample_list)
DBsample1_clean = drop_noise(DBsample1, DBsample_list)
DBsample2_clean = drop_noise(DBsample2, DBsample_list)
DBsample3_clean = drop_noise(DBsample3, DBsample_list)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
DBsample3_clean.head()

,feat_257,feat_269,feat_308,feat_315,feat_336,feat_341,feat_395,feat_504,feat_526,feat_639,...,feat_701,feat_724,feat_736,feat_769,feat_808,feat_829,feat_867,feat_920,feat_956,target
2000,0.077185,0.724953,1.451919,0.222816,-2.671703,-2.788843,-2.930196,-2.776659,1.894507,-2.579466,...,-2.106660,1.959709,-2.105096,0.987539,-0.173542,-1.309707,1.977380,-0.560210,-1.713323,0
2001,-0.458983,6.315434,-0.263639,3.464669,1.334163,4.482667,2.553172,0.189120,0.705362,-0.480131,...,-5.165145,-3.380224,2.416505,-1.960566,-0.581636,3.073353,1.683694,-3.315520,-0.651008,0
2002,-2.931058,2.267584,-1.754548,2.593913,-0.587585,0.113437,2.734551,-0.643102,0.170425,-2.114823,...,-1.677454,-2.761319,2.006122,-0.885269,0.963534,-2.755311,0.476961,0.912386,-2.474002,0
2003,-1.477788,-3.664489,-0.695112,-0.294983,-2.190510,-3.020389,-0.203724,1.897029,-0.034787,-0.392432,...,1.912592,-0.000522,-1.023399,2.343162,1.995171,-2.187725,0.183902,1.256132,-0.813963,1
2004,2.061357,-0.859231,-1.821543,-0.334201,0.997277,-1.525705,-2.155599,-0.842014,2.211915,0.024668,...,-0.617800,3.551031,0.358455,1.983522,-2.079605,1.548041,-0.461750,0.044153,0.160922,1


In [39]:
# save noiseless/clean samples and noisy samples.  what a relief!

pickle.dump(UCIsample1_clean, open('UCIsample1_clean', "wb" ) )
pickle.dump(UCIsample2_clean, open('UCIsample2_clean', "wb" ) )
pickle.dump(UCIsample3_clean, open('UCIsample3_clean', "wb" ) )
pickle.dump(DBsample1_clean, open('DBsample1_clean', "wb" ) )
pickle.dump(DBsample2_clean, open('DBsample2_clean', "wb" ) )
pickle.dump(DBsample3_clean, open('DBsample3_clean', "wb" ) )

